In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import ecc, musig

In [ ]:
# Example ScriptPath Spend
from ecc import PrivateKey, S256Point
from hash import sha256
from script import address_to_script_pubkey
from taproot import TapScript, TapLeaf, TapBranch
from tx import Tx, TxIn, TxOut
from witness import Witness
my_secret = 21000000
my_private_key = PrivateKey(my_secret)
p = S256Point.parse(bytes.fromhex("cd04c1bf88ca891af152fc57c36523ab59efb16b7ec07caca0cfc4a1f2051d9e"))
pubkey_2 = bytes.fromhex("331a8f6a14e1b41a6b523ddb505fbc0662a6446bd42408692497297d3474aeec")
pubkey_3 = bytes.fromhex("158a49d62c384c539a453e41a70214cfb85184954ae5c8df4b47eb74d58ff16f")
my_xonly = my_private_key.point.xonly()
tap_script_1 = TapScript([my_xonly, 0xAC])
tap_leaf_1 = TapLeaf(tap_script_1)
tap_script_2 = TapScript([pubkey_2, 0xAC])
tap_leaf_2 = TapLeaf(tap_script_2)
tap_script_3 = TapScript([pubkey_3, 0xAC])
tap_leaf_3 = TapLeaf(tap_script_3)
prev_tx = bytes.fromhex("42251c52fa8c6a6a349eee1729ced9587483ea6d6e210d6c42bb640d33a4da25")
prev_index = 0
target_address = "tb1pxh7kypwsvxnat0z6588pufhx43r2fnqjyn846qj5kx8mgqcamvjsyn5cjg"
fee = 500
tap_branch_1 = TapBranch(tap_leaf_1, tap_leaf_2)
tap_branch_2 = TapBranch(tap_branch_1, tap_leaf_3)
tx_in = TxIn(prev_tx, prev_index)
target_amount = tx_in.value(network="signet") - fee
target_script = address_to_script_pubkey(target_address)
tx_out = TxOut(target_amount, target_script)
tx_obj = Tx(1, [tx_in], [tx_out], network="signet", segwit=True)
cb = tap_branch_2.control_block(p, tap_leaf_1)
tx_in.witness = Witness([tap_script_1.raw_serialize(), cb.serialize()])
msg = tx_obj.sig_hash(0)
sig = my_private_key.sign_schnorr(msg).serialize()
tx_in.witness.items.insert(0, sig)
tx_obj.verify()
print(tx_obj.serialize().hex())

### Exercise 1

You have been sent 100,000 sats to the address you created in the last exercise of the last session. Use the TapScript with your pubkey to spend all of it and send everything minus the fee to <code>tb1q7kn55vf3mmd40gyj46r245lw87dc6us5n50lrg</code>. Use <a href="https://mempool.space/signet/tx/push to broadcast your transaction" target="_mempool">Mempool Signet</a> to broadcast your transaction.



In [ ]:
# Exercise 1

from ecc import PrivateKey, S256Point
from hash import sha256
from helper import big_endian_to_int
from script import address_to_script_pubkey
from taproot import TapScript, TapLeaf, TapBranch
from tx import Tx, TxIn, TxOut
from witness import Witness
my_email = b"<fill this in with your email>"
my_secret = big_endian_to_int(sha256(my_email))
my_private_key = PrivateKey(my_secret)
p = S256Point.parse(bytes.fromhex("cd04c1bf88ca891af152fc57c36523ab59efb16b7ec07caca0cfc4a1f2051d9e"))
pubkey_2 = bytes.fromhex("331a8f6a14e1b41a6b523ddb505fbc0662a6446bd42408692497297d3474aeec")
pubkey_3 = bytes.fromhex("158a49d62c384c539a453e41a70214cfb85184954ae5c8df4b47eb74d58ff16f")
my_xonly = my_private_key.point.xonly()
tap_script_1 = TapScript([my_xonly, 0xAC])
tap_leaf_1 = TapLeaf(tap_script_1)
tap_script_2 = TapScript([pubkey_2, 0xAC])
tap_leaf_2 = TapLeaf(tap_script_2)
tap_script_3 = TapScript([pubkey_3, 0xAC])
tap_leaf_3 = TapLeaf(tap_script_3)
prev_tx = bytes.fromhex("<fill this in with the tx hash of a UTXO that belongs to you>")
# fill in the following with the correct output index from the UTXO
prev_index = -1
target_address = "tb1q7kn55vf3mmd40gyj46r245lw87dc6us5n50lrg"
fee = 500
# create the two branches needed (leaf 1, leaf 2), (branch 1, leaf 3)


# create a transaction input

# calculate the target amount

# calculate the target script

# create a transaction output

# create a transaction, segwit=True and network="signet"

# create the control block from the TapBranch control_block method with internal_pubkey and tap_leaf_1

# set the tx_in's witness to be the tap_script_1.raw_serialize() and control block, serialized

# set the message to be the transaction's sig_hash at index 0

# use sign_schnorr with yoru private key on the message and serialize it to get the signature

# insert the sig in front of the other elements in the witness using tx_in.witness.items.insert

# verify the transaction

# print the serialized hex


# MuSig2 Key Aggregation
* Produce a Schnorr Signature $(R,s)$ from a group of $n$ public keys $P_1, P_2,..., P_n$ and present a single public key $P$ to satisfy $sG-\mathcal{H}(R,P,z)P=R$
* Each participant has its own secret $e_iG=P_i$
* We create a Group Commitment $L=\mathcal{H}(P_1||P_2||...||P_n)$
* Each Public Key gets its own keyagg coefficient: $\mathcal{H}(L,P_i)$
* The Group Public Key is $P=\mathcal{H}(L,P_1)P_1+\mathcal{H}(L,P_2)P_2+...+\mathcal{H}(L,P_n)P_n$


In [ ]:
# creating a group public key
from ecc import S256Point
from hash import hash_keyagglist, hash_keyaggcoef
from helper import big_endian_to_int
raw_pubkeys = ["02ed3bace23c5e17652e174c835fb72bf53ee306b3406a26890221b4cef7500f88", "03cd5a3be41717d65683fe7a9de8ae5b4b8feced69f26a8b55eeefbcc2e74b75fb", "0385a7b790fc9d962493788317e4874a4ab07f1e9c78c773c47f2f6c96df756f05"]
pubkeys = [S256Point.parse(bytes.fromhex(r)) for r in raw_pubkeys]
preimage = b""
for pubkey in pubkeys:
    preimage += pubkey.sec()
group_commitment = hash_keyagglist(preimage)
terms = []
for p_i in pubkeys:
    h = hash_keyaggcoef(group_commitment + p_i.sec())
    c_i = big_endian_to_int(h)
    terms.append(c_i * p_i)
p = S256Point.sum(terms)
print(p.sec().hex())

### Exercise 2

Create a new group public key with these component public keys:

* 034a5169f673aa632f538aaa128b6348536db2b637fd89073d49b6a23879cdb3ad
* 0225fa6a4190ddc87d9f9dd986726cafb901e15c21aafd2ed729efed1200c73de8
* 03ed214e8ce499d92a2085e7e6041b4f081c7d29d8770057fc705a131d2918fcdb
* 02609ae8d31e3b290e74483776c1c8dfc2756b87d9635d654eb9e1ca95c228b169
* 02ffe558e388852f0120e46af2d1b370f85854a8eb0841811ece0e3e03d282d57c
* 02d42d696f2c343dc67d80fcd85dbbdb2edef3cac71126625d0cbcacc231a00015



In [ ]:
# Exercise 2

from ecc import S256Point
from hash import hash_keyagglist, hash_keyaggcoef
from helper import big_endian_to_int
raw_pubkeys = ["034a5169f673aa632f538aaa128b6348536db2b637fd89073d49b6a23879cdb3ad", "0225fa6a4190ddc87d9f9dd986726cafb901e15c21aafd2ed729efed1200c73de8", "03ed214e8ce499d92a2085e7e6041b4f081c7d29d8770057fc705a131d2918fcdb", "02609ae8d31e3b290e74483776c1c8dfc2756b87d9635d654eb9e1ca95c228b169", "02ffe558e388852f0120e46af2d1b370f85854a8eb0841811ece0e3e03d282d57c", "02d42d696f2c343dc67d80fcd85dbbdb2edef3cac71126625d0cbcacc231a00015"]
# create the pubkeys using S256Point.parse and bytes.fromhex()

# The preimage for the group commitment is L=H(P_1||P_2||...||P_n) in sec format

# loop through the pubkeys

    # add the sec for this key to the preimage

# create the commitment hash by doing hash_keyagglist on the preimage

# create a list for the terms that will get summed

# loop through the pubkeys

    # calculate the hash of the group commitment and the sec usig hash_keyaggcoef

    # convert the hash to an integer

    # add the coefficient * pubkey to the list of terms

# the group pubkey is the S256Point.sum of the terms

# print the group pubkey's sec in hex


# BIP327 Key Aggregation
* The coefficients would work fine, but there's one optimization made in BIP327
* Specifically, the second coefficient is always 1, all other coefficients stay the same
* The reason that it's not the first is because of a corner case where all the public keys are the same
* There's also the fact that there's the possibility of the group point being odd
* The way this is solved is by every participant negating their secret


In [ ]:
# creating a BIP327 group public key
from ecc import S256Point
from hash import hash_keyagglist, hash_keyaggcoef
from helper import big_endian_to_int
raw_pubkeys = ["02ed3bace23c5e17652e174c835fb72bf53ee306b3406a26890221b4cef7500f88", "03cd5a3be41717d65683fe7a9de8ae5b4b8feced69f26a8b55eeefbcc2e74b75fb", "0385a7b790fc9d962493788317e4874a4ab07f1e9c78c773c47f2f6c96df756f05"]
pubkeys = [S256Point.parse(bytes.fromhex(r)) for r in raw_pubkeys]
preimage = b""
for pubkey in pubkeys:
    preimage += pubkey.sec()
group_commitment = hash_keyagglist(preimage)
terms = []
second_point = None
for p_i in pubkeys:
    if p_i != pubkeys[0] and second_point is None:
        second_point = p_i
    if p_i == second_point:
        c_i = 1
    else:
        h = hash_keyaggcoef(group_commitment + p_i.sec())
        c_i = big_endian_to_int(h)
    terms.append(c_i * p_i)
p = S256Point.sum(terms)
print(p.sec().hex())

### Exercise 3

Create a BIP327 group public key with these public keys:

* 034a5169f673aa632f538aaa128b6348536db2b637fd89073d49b6a23879cdb3ad
* 0225fa6a4190ddc87d9f9dd986726cafb901e15c21aafd2ed729efed1200c73de8
* 03ed214e8ce499d92a2085e7e6041b4f081c7d29d8770057fc705a131d2918fcdb
* 02609ae8d31e3b290e74483776c1c8dfc2756b87d9635d654eb9e1ca95c228b169
* 02ffe558e388852f0120e46af2d1b370f85854a8eb0841811ece0e3e03d282d57c
* 02d42d696f2c343dc67d80fcd85dbbdb2edef3cac71126625d0cbcacc231a00015



In [ ]:
# Exercise 3

from ecc import S256Point
from hash import hash_keyagglist, hash_keyaggcoef
from helper import big_endian_to_int
raw_pubkeys = ["034a5169f673aa632f538aaa128b6348536db2b637fd89073d49b6a23879cdb3ad", "0225fa6a4190ddc87d9f9dd986726cafb901e15c21aafd2ed729efed1200c73de8", "03ed214e8ce499d92a2085e7e6041b4f081c7d29d8770057fc705a131d2918fcdb", "02609ae8d31e3b290e74483776c1c8dfc2756b87d9635d654eb9e1ca95c228b169", "02ffe558e388852f0120e46af2d1b370f85854a8eb0841811ece0e3e03d282d57c", "02d42d696f2c343dc67d80fcd85dbbdb2edef3cac71126625d0cbcacc231a00015"]
# create the pubkeys using S256Point.parse and bytes.fromhex()

# The preimage for the group commitment is L=H(P_1||P_2||...||P_n) in sec format

# loop through the pubkeys

    # add the sec for this key to the preimage

# create the commitment hash by doing hash_keyagglist on the preimage

# create a list for the terms that will get summed

# initialize the second point with None

# loop through the pubkeys

    # if the second point is None and the pubkey is not the first one

        # then designate the second point to be this pubkey

    # if the current pubkey is the second point, coefficient is 1


    # otherwise

        # calculate the hash_keyaggcoef of the group commitment and the sec

        # convert the hash to an integer

    # add the coefficient * pubkey to the list of terms

# the group pubkey is the S256Point.sum of the terms

# print the group pubkey's sec in hex


### Exercise 4




#### Make [this test](/edit/session3/musig.py) pass: `musig.py:KeyAggTest:test_compute_group_point`

In [ ]:
# Exercise 4

reload(musig)
run(musig.KeyAggTest('test_compute_group_point'))

### Exercise 5

Create a 2-of-2 BIP327 public key sharing a key with your neighbor.



In [ ]:
# Exercise 5

from ecc import PrivateKey, S256Point
from helper import big_endian_to_int, sha256
from musig import KeyAggregator
my_email = b"<fill this in with your email>"
my_secret = big_endian_to_int(sha256(my_email))
my_pubkey = PrivateKey(my_secret).point
# print your pubkey in sec format

# get your neighbor's sec pubkey
neighbor_pubkey = S256Point.parse(bytes.fromhex("<your neighbor's sec pubkey>"))
# create the keyaggcontext

# now print the group point's sec in hex


# MuSig2 Nonce Creation
* Every participant creates a private nonce share (numbers $l_i$ and $m_i$) and communicates the nonce share points to the coordinator ($l_i * G=S_i, m_i * G=T_i$)
* When the nonce share points are gathered, the coordinator communicates nonce point sums to each participant. $S = S_1+S_2+...+S_n, T=T_1+T_2+...+T_n$ along with the message being signed and the key aggregation data.
* The nonce coefficient is $b = \mathcal{H}(S || T || P || z)$ where $P$ is the group point and $z$ is the message being signed
* The nonce point for the signature $R = S + b * T$
* The same nonce coefficient ($b$) can now be used to determine each participant's nonce ($k$) $k_i = l_i + b * m_i$


In [ ]:
# Example nonce generation
from ecc import N, PrivateKey
from hash import hash_musignoncecoef
from musig import KeyAggregator, MuSigParticipant, MuSigCoordinator
msg = b"MuSig2 is awesome!"
participants = [MuSigParticipant(PrivateKey(i * 1000)) for i in range(1, 7)]
pubkeys = [p.point for p in participants]
coor = MuSigCoordinator(pubkeys)
for p in participants:
    nonce_share = p.generate_nonce_share(msg=msg, rand=b'')
    coor.register_nonce_share(p.point.sec(), nonce_share)
group_point = coor.keyagg.group_point
s = S256Point.sum([n.s for n in coor.nonce_shares.values()])
t = S256Point.sum([n.t for n in coor.nonce_shares.values()])
h = hash_musignoncecoef(s.sec()+t.sec()+group_point.xonly()+msg)
b = big_endian_to_int(h)
r = s + b*t
print(r.sec().hex())
k = (participants[0].nonce_private_share.l + b * participants[0].nonce_private_share.m) % N
print(hex(k))

### Exercise 6

Calculate the k for participant 1:

Participant 1's secret: 1000
Participant 2's secret: 2000

message: b"Hello World!"

Participant 1's $l$ and $m$: 3000, 4000
Participant 2's $l$ and $m$: 5000, 6000



In [ ]:
# Exercise 6

from ecc import N, PrivateKey
from hash import hash_musignoncecoef
from helper import big_endian_to_int
from musig import KeyAggregator, NoncePrivateShare, MuSigParticipant, MuSigCoordinator
participant_1 = MuSigParticipant(PrivateKey(1000))
participant_2 = MuSigParticipant(PrivateKey(2000))
z = b"Hello World!"
nonce_share_1 = NoncePrivateShare(3000, 4000, participant_1.point)
nonce_share_2 = NoncePrivateShare(5000, 6000, participant_2.point)
pubkeys = [participant_1.point, participant_2.point]
coor = MuSigCoordinator(pubkeys)
coor.register_nonce_share(participant_1.point.sec(), nonce_share_1.public_share)
coor.register_nonce_share(participant_2.point.sec(), nonce_share_2.public_share)
# grab the group point, p, from coordinator's group_point

# calculate s and t by summing the s and t properties from the nonce_shares.values()


# calculate the hash_musignoncecoef of s's sec, t's sec, p's xonly and the message, z

# the nonce coefficient, b, is the hash interpreted as a big endian integer

# the r=S+bT

# print the nonce point's sec in hex

# k=l+bm (l and m are properties of nonce_share_1) make sure to mod by N

# print the hex of k


### Exercise 7




#### Make [this test](/edit/session3/musig.py) pass: `musig.py:NonceAggTest:test_compute_nonce_point`

In [ ]:
# Exercise 7

reload(musig)
run(musig.NonceAggTest('test_compute_nonce_point'))

### Exercise 8

Create a nonce point and nonce coefficient with your neighbor for this message: b"Love thy neighbor"



In [ ]:
# Exercise 8

from ecc import N, PrivateKey
from hash import hash_musignoncecoef
from helper import big_endian_to_int, sha256
from musig import NoncePrivateShare, NoncePublicShare, MuSigParticipant, MuSigCoordinator
msg = b"Love thy neighbor"
my_secret = big_endian_to_int(sha256(b"<my email address>"))
me = MuSigParticipant(PrivateKey(my_secret))
my_pubkey = me.point
neighbor_pubkey = S256Point.parse(bytes.fromhex("<my neighbor's sec pubkey>"))
# create the coordinator with me and my neighbor

# generate my nonce share using generate_nonce_share

# print the nonce share's serialization in hex and share with your neighbor

neighbor_share = NoncePublicShare.parse(bytes.fromhex("<fill in with your neighbor's hex public nonce share>"))
# register my nonce share to the coordinator

# register neighbor's share to the coordinator

# compute the nonce point from the coordinator

# print the sec format in hex of the nonce point


# MuSig2 Partial Signatures
* Every participant calculates $k_i=l_i+b m_i$ using the nonce coefficient $b= \mathcal{H}(S||T||P||z)$
* They also need the keyagg coefficient $c_i = \mathcal{H}(L || P_i)$ and the challenge $d=\mathcal{H}(R,P,z)$ where $z$ is the message)
* The partial signature can then be calculated $s_i = k_i + c_i * d * e_i$ where $e_i$ is the secret
* The sum of the partial signatures is the s part of the Schnorr Signature $s=s_1+s_2+...+s_n$
* If $R$ is odd, we use $-k_i$ for the signing. Similarly, if $P$ is odd, we use $-e_i$ for signing


In [ ]:
from ecc import N, PrivateKey
from musig import SigningContext, MuSigParticipant, MuSigCoordinator
participant_1 = MuSigParticipant(PrivateKey(1000))
participant_2 = MuSigParticipant(PrivateKey(2000))
msg = b"MuSig2 is awesome!"
nonce_share_1 = participant_1.generate_nonce_share(msg=msg, rand=b'')
nonce_share_2 = participant_2.generate_nonce_share(msg=msg, rand=b'')
pubkeys = [participant_1.point, participant_2.point]
coor = MuSigCoordinator(pubkeys)
coor.register_nonce_share(participant_1.point.sec(), nonce_share_1)
coor.register_nonce_share(participant_2.point.sec(), nonce_share_2)
context = coor.create_signing_context(msg)
if context.nonce_point.even:
    k = participant_1.nonce(context.nonce_coef)
else:
    k = N - participant_1.nonce(context.nonce_coef)
if context.group_point.even:
    e = participant_1.private_key.secret
else:
    e = N - participant_1.private_key.secret
c = context.keyagg_coef(participant_1.point)
d = context.challenge
s = (k + c * d * e) % N
print(hex(s))

# MuSig2 Partial Signature Verification
* To verify a partial signature, we need some data from the coordinator: group commitment, message being signed, aggregate nonce point and aggregate pubkey
* We also need the participant nonce point and pubkey.
* $c_i$ is the aggregate key coefficient $\mathcal{H}(L||P_i)$ and $d$ is group commitment $\mathcal{H}(R||P||z)$
* $s_i=k_i+c_i d e_i$ so what we check is $s_i G=k_iG+c_i d e_i G=R_i+c_i d P_i$ or $R=s_i G-c_i d P_i$


In [ ]:
# example of verifying a partial signature
from ecc import G, N, PrivateKey
from musig import SigningContext, MuSigParticipant, MuSigCoordinator
participant_1 = MuSigParticipant(PrivateKey(1000))
participant_2 = MuSigParticipant(PrivateKey(2000))
msg = b"MuSig2 is awesome!"
nonce_share_1 = participant_1.generate_nonce_share(msg=msg, rand=b'')
nonce_share_2 = participant_2.generate_nonce_share(msg=msg, rand=b'')
pubkeys = [participant_1.point, participant_2.point]
coor = MuSigCoordinator(pubkeys)
coor.register_nonce_share(participant_1.point.sec(), nonce_share_1)
coor.register_nonce_share(participant_2.point.sec(), nonce_share_2)
context = coor.create_signing_context(msg)
s = 0xa5aa0ae6ba94c1d8948929b7422d14869476011f4a5904c8f46b276396d4051a
if context.nonce_point.even:
    r = nonce_share_1.nonce_point(context.nonce_coef)
else:
    r = -1 * nonce_share_1.nonce_point(context.nonce_coef)
if context.group_point.even:
    p = participant_1.point
else:
    p = -1 * participant_1.point
c = context.keyagg_coef(participant_1.point)
d = context.challenge
print(s * G == r + c * d * p)

### Exercise 9

Calculate the partial signature for participant 2:

Participant 1's secret: 1000
Participant 2's secret: 2000

message: b"Hello World!"

Participant 1's $l$ and $m$: 3000, 4000
Participant 2's $l$ and $m$: 5000, 6000



In [ ]:
# Exercise 9

from ecc import N, PrivateKey
from musig import SigningContext, MuSigParticipant, MuSigCoordinator
participant_1 = MuSigParticipant(PrivateKey(1000))
participant_2 = MuSigParticipant(PrivateKey(2000))
msg = b"Hello World!"
nonce_share_1 = NoncePrivateShare(3000, 4000, participant_1.point)
nonce_share_2 = NoncePrivateShare(5000, 6000, participant_2.point)
participant_1.nonce_private_share = nonce_share_1
participant_2.nonce_private_share = nonce_share_2
pubkeys = [participant_1.point, participant_2.point]
coor = MuSigCoordinator(pubkeys)
coor.register_nonce_share(participant_1.point.sec(), nonce_share_1.public_share)
coor.register_nonce_share(participant_2.point.sec(), nonce_share_2.public_share)
# create the signing context, which should aggregate the points

# determine the second participant's nonce (k_i) from the nonce point's evenness




# determine the second participant's secret (e_i) from the group point's evenness




# use the context's keylagg_coef method to get the keyagg coefficient (c_i = H(L||P_i))

# use the context's challenge method to get the group challenge (d = H(R||P||z))

# now get the partial signature s_i = k + c_i * d * e_i

# print the hex of the partial signature


### Exercise 10

Verify the partial signature for participant 2



In [ ]:
# Exercise 10

from ecc import G, N, PrivateKey
from musig import SigningContext, MuSigParticipant, MuSigCoordinator
participant_1 = MuSigParticipant(PrivateKey(1000))
participant_2 = MuSigParticipant(PrivateKey(2000))
msg = b"Hello World!"
nonce_share_1 = NoncePrivateShare(3000, 4000, participant_1.point)
nonce_share_2 = NoncePrivateShare(5000, 6000, participant_2.point)
participant_1.nonce_private_share = nonce_share_1
participant_2.nonce_private_share = nonce_share_2
pubkeys = [participant_1.point, participant_2.point]
coor = MuSigCoordinator(pubkeys)
coor.register_nonce_share(participant_1.point.sec(), nonce_share_1.public_share)
coor.register_nonce_share(participant_2.point.sec(), nonce_share_2.public_share)
# fill in what s equals from the last exercise

# create the signing context, which should aggregate the points

# determine the second participant's nonce point (R_i) from the nonce point's evenness




# determine the second participant's pubkey (P_i) from the group point's evenness




# get the keyagg coefficient (c) for the second participant

# get the challenge for the group (d)

# check if s_i * G == R + c * d * P


### Exercise 11

Sum the partial signatures, create a Schnorr Signature and verify it using the group point



In [ ]:
# Exercise 11

from ecc import G, N, PrivateKey, SchnorrSignature
from musig import SigningContext, MuSigParticipant, MuSigCoordinator
participant_1 = MuSigParticipant(PrivateKey(1000))
participant_2 = MuSigParticipant(PrivateKey(2000))
msg = b"Hello World!"
nonce_share_1 = NoncePrivateShare(3000, 4000, participant_1.point)
nonce_share_2 = NoncePrivateShare(5000, 6000, participant_2.point)
participant_1.nonce_private_share = nonce_share_1
participant_2.nonce_private_share = nonce_share_2
pubkeys = [participant_1.point, participant_2.point]
coor = MuSigCoordinator(pubkeys)
coor.register_nonce_share(participant_1.point.sec(), nonce_share_1.public_share)
coor.register_nonce_share(participant_2.point.sec(), nonce_share_2.public_share)
context = coor.create_signing_context(msg)
s_1 = 0x1aad95d9490e4b8599377ff6a546a1d075fb4242c749dbcbc010589e23c21776
s_2 = 0x9d4815237aa9e06f5ae095bef95898c584f835c9df0389ad46e1f94c8034b621
# sum the two partial sigs and mod by N

# get the nonce point from the context

# create the Schnorr Signature using the r and the s

# check the validity of the schnorr signature using the group point from the context


### Exercise 12




#### Make [this test](/edit/session3/musig.py) pass: `musig.py:PartialSigTest:test_verify`

In [ ]:
# Exercise 12

reload(musig)
run(musig.PartialSigTest('test_verify'))

### Exercise 13




#### Make [this test](/edit/session3/musig.py) pass: `musig.py:PartialSigTest:test_sign`

In [ ]:
# Exercise 13

reload(musig)
run(musig.PartialSigTest('test_sign'))

### Exercise 14

Trade partial signatures with your neighbor and verify for the message from Exercise 10.



In [ ]:
# Exercise 14

from ecc import N, PrivateKey
from hash import hash_musignoncecoef
from helper import big_endian_to_int, sha256
from musig import NoncePrivateShare, NoncePublicShare, MuSigParticipant, MuSigCoordinator
msg = b"Love thy neighbor"
my_secret = big_endian_to_int(sha256(b"<my email address>"))
me = MuSigParticipant(PrivateKey(my_secret))
my_pubkey = me.point
neighbor_pubkey = S256Point.parse(bytes.fromhex("<my neighbor's sec pubkey>"))
# create the coordinator with me and my neighbor

# generate my nonce share using generate_nonce_share

# print the nonce share's serialization in hex and share with your neighbor

 neighbor_share = NoncePublicShare.parse(bytes.fromhex("<fill in with your neighbor's hex public nonce share>"))
# register my nonce share to the coordinator

# register neighbor's share to the coordinator

# create the signing context

# sign with the context

# print the partial sig in hex to share with your neighbor

neighbor_sig = bytes.fromhex("<fill in with your neighbor's partial sig in hex>")
# sum the two partial sigs converted to integers and then mod by N

# get the nonce point from the context

# create the Schnorr Signature using the r and the s

# check the validity of the schnorr signature using the group point from the context


# MuSig2 Group Point Tweaking
* If the MuSig2 group point is the KeyPath Spend, then there is a tweak $t$
* The group point $P$ and tweak $t$ make the external pubkey $Q=P+tG$
* $Q$ is $x$-only, so that determines $e_i$ negation, not the $P$
* We set $Q$ to be the group point


In [ ]:
# example of tweaking the MuSig2 group pubkey
from ecc import G, PrivateKey
from hash import hash_taptweak
from helper import big_endian_to_int
from musig import KeyAggregator
pubkeys = [PrivateKey(i * 1000).point for i in range(1,7)]
keyagg = KeyAggregator(pubkeys)
merkle_root = b""
tweak = hash_taptweak(keyagg.group_point.xonly() + merkle_root)
t = big_endian_to_int(tweak)
keyagg.group_point = keyagg.group_point + t * G
print(keyagg.group_point.sec().hex())

# Partial Sig Aggregation for even/odd $Q$
* For even $Q$: the Schnorr Signature $(R, s+td)$ will validate for the tweaked key $Q$
* For odd $Q$: The Schnorr Signature $(R, s-td)$ will validate for the tweaked key $-Q$


In [ ]:
# example of aggregating a tweaked group pubkey
from ecc import G, N, PrivateKey, SchnorrSignature
from helper import big_endian_to_int
from musig import SigningContext, MuSigParticipant, MuSigCoordinator
participant_1 = MuSigParticipant(PrivateKey(1000))
participant_2 = MuSigParticipant(PrivateKey(2000))
msg = b"Hello World!"
nonce_share_1 = NoncePrivateShare(3000, 4000, participant_1.point)
nonce_share_2 = NoncePrivateShare(5000, 6000, participant_2.point)
participant_1.nonce_private_share = nonce_share_1
participant_2.nonce_private_share = nonce_share_2
pubkeys = [participant_1.point, participant_2.point]
coor = MuSigCoordinator(pubkeys)
tweak = hash_taptweak(coor.keyagg.group_point.xonly() + b"")
t = big_endian_to_int(tweak)
coor.keyagg.group_point = coor.keyagg.group_point + t * G
coor.register_nonce_share(participant_1.point.sec(), nonce_share_1.public_share)
coor.register_nonce_share(participant_2.point.sec(), nonce_share_2.public_share)
context = coor.create_signing_context(msg)
s_1 = big_endian_to_int(participant_1.sign(context))
s_2 = big_endian_to_int(participant_2.sign(context))
s = (s_1 + s_2) % N
d = context.challenge
if context.group_point.even:
    s = (s + d * t) % N
else:
    s = (s - d * t) % N
r = context.nonce_point
sig = SchnorrSignature(r, s)
print(context.group_point.verify_schnorr(msg, sig))

### Exercise 15

Sum the partial signatures, create a Schnorr Signature and verify it using the group point



In [ ]:
# Exercise 15

from ecc import G, N, PrivateKey, SchnorrSignature
from musig import SigningContext, MuSigParticipant, MuSigCoordinator
participant_1 = MuSigParticipant(PrivateKey(1000))
participant_2 = MuSigParticipant(PrivateKey(2000))
msg = b"Hello World!"
nonce_share_1 = NoncePrivateShare(3000, 4000, participant_1.point)
nonce_share_2 = NoncePrivateShare(5000, 6000, participant_2.point)
participant_1.nonce_private_share = nonce_share_1
participant_2.nonce_private_share = nonce_share_2
pubkeys = [participant_1.point, participant_2.point]
merkle_root = bytes.fromhex("818c9d665b78324ba673afca23f5f4f5512214ccfd0554fe82c5f99f5a29689a")
coor = MuSigCoordinator(pubkeys, merkle_root=merkle_root)
t = coor.keyagg.tweak_amount
coor.register_nonce_share(participant_1.point.sec(), nonce_share_1.public_share)
coor.register_nonce_share(participant_2.point.sec(), nonce_share_2.public_share)
context = coor.create_signing_context(msg)
s_1 = 0xa23b11cb8c2120ac414f15fc27f67f4588f770b1e3a2012406a8771287410da1
s_2 = 0x22d4db66c5a3bfecf63ac2b6402a5cba717938d33006a0ee8b1d398aee9dda1e
# sum the two partial sigs and mod by N

# get the challenge from the context

# add d*t to s if group point is even, subtract d*t from s if odd




# get the nonce point from the context

# create the Schnorr Signature using the r and the s

# check the validity of the schnorr signature using the group point from the context


### Exercise 16




#### Make [this test](/edit/session3/musig.py) pass: `musig.py:PartialSigTest:test_compute_sig`

In [ ]:
# Exercise 16

reload(musig)
run(musig.PartialSigTest('test_compute_sig'))

### Exercise 17

Trade partial signatures with your neighbor and verify for the message from Exercise 10.



In [ ]:
# Exercise 17

from ecc import N, PrivateKey
from hash import hash_musignoncecoef
from helper import big_endian_to_int, sha256
from musig import NoncePrivateShare, NoncePublicShare, MuSigParticipant, MuSigCoordinator
msg = b"Love thy neighbor"
my_secret = big_endian_to_int(sha256(b"<my email address>"))
me = MuSigParticipant(PrivateKey(my_secret))
my_pubkey = me.point
Neighbor_pubkey = S256Point.parse(bytes.fromhex("<my neighbor's sec pubkey>"))
merkle_root = bytes.fromhex("818c9d665b78324ba673afca23f5f4f5512214ccfd0554fe82c5f99f5a29689a")
# create the coordinator with my pubkey and my neighbor's pubkey with the merkle root

# generate my nonce share using generate_nonce_share

# print the nonce share's serialization in hex and share with your neighbor

neighbor_share = NoncePublicShare.parse(bytes.fromhex("<fill in with your neighbor's hex public nonce share>"))
# register my nonce share to the coordinator

# register neighbor's share to the coordinator

# create the signing context

# sign with the context

# register my partial signature with the coordinator

# print the partial sig in hex to share with your neighbor

neighbor_sig = bytes.fromhex("<fill in with your neighbor's partial sig in hex>")
# register neighbor's sig with the coordinator

# get the schnorr signature from the coordinator

# print whether the signature verifies

# print the signature, serialized in hex


### Exercise 18

Make an address with your neighbor where the internal pubkey is a 2-of-2 MuSig and the single TapLeaf is also a 2-of-2 MuSig

Submit your address at [this link](https://docs.google.com/spreadsheets/d/1BHqFAzgfThrf64q9pCinwTd7FitJrL5Is3HHBR3UyeI/edit?usp=sharing)




In [ ]:
# Exercise 18

from ecc import N, PrivateKey
from hash import hash_musignoncecoef
from helper import big_endian_to_int, sha256
from musig import KeyAggregator, MuSigParticipant, MuSigCoordinator
from taproot import TapLeaf, TapScript
my_secret = big_endian_to_int(sha256(b"<my email address>"))
me = MuSigParticipant(PrivateKey(my_secret))
neighbor_pubkey = S256Point.parse(bytes.fromhex("<my neighbor's sec pubkey>"))
# collect the pubkeys in a list

# use the KeyAggregator to get the pubkey

# the TapScript we want is [P, 0xAC] where P is the group point in xonly, 0xAC is OP_CHECKSIG

# create a TapLeaf with the single TapScript

# set the merkle root to be the hash of this TapLeaf

# create the coordinator with pubkeys with the merkle root

# use the address method to get the p2tr address on signet
